## Setup: Agent Config and Tool Definition

In [3]:
from typing import Literal
import os
from tavily import TavilyClient
from langchain.agents import create_agent
from langchain.agents.middleware.types import AgentMiddleware
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command, interrupt
from deepagents.graph import create_deep_agent 
from deepagents.middleware.subagents import CompiledSubAgent
from dotenv import load_dotenv

load_dotenv()

tavily_client = TavilyClient(os.getenv("TAVILY_API_KEY"))
model = ChatOpenAI(model="gpt-5-nano", temperature=0)

@tool
def get_current_weather(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """
    A tool to search the web for current weather conditions.
    Args:
        query: The query about weather to search for
        max_results: The maximum number of results to return.
        topic: The topic of the search.
        include_raw_content: Whether to include the raw content of the search results.
    """
    return tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )

class WeatherApprovalMiddleware(AgentMiddleware):
    """Pause the weather subagent until a human approves the tool output."""

    def before_model(self, state):
        messages = state['messages']

        last_message = messages[-1]
        if not isinstance(last_message, ToolMessage) or last_message.name != "get_current_weather":
            return None

        interrupt_payload = {
            "prompt": "Approve or edit the latest weather lookup result.",
            "tool_call_id": last_message.tool_call_id,
            "tool_name": last_message.name,
            "tool_output": last_message.content,
        }

        decision = interrupt(interrupt_payload)

        if not isinstance(decision, dict):
            raise ValueError(
                "Resume payload must be a dict with 'status' and optional 'message'."
            )

        status = decision.get("status")
        if status not in {"approved", "edit"}:
            raise ValueError("Resume payload must include status 'approved' or 'edit'.")

        review_note = decision.get("message")
        if status == "edit" and not review_note:
            raise ValueError("Edits must include a 'message' detailing the edit to make.")
        if not review_note:
            review_note = "Weather lookup approved."

        human_msg = HumanMessage(
            content=f"[{"APPROVED" if status == "approved" else "EDITED"}] {review_note}",
            name="tool_reviewer",
            additional_kwargs={
                "approval_status": status,
                "tool_call_id": last_message.tool_call_id
            },
        )
        return {"messages": [human_msg]}


weather_agent_runnable = create_agent(
    model,
    system_prompt='You are a weather expert. Use the get_current_weather tool to get the current weather conditions for a given location.',
    tools=[get_current_weather],
    middleware=[WeatherApprovalMiddleware()],
    checkpointer=True, # 
)


weather_agent = CompiledSubAgent(
    name='weather-agent',
    description='A weather expert that uses the get_current_weather tool to get the current weather conditions for a given location.',
    runnable=weather_agent_runnable,
)


super_agent_ckpt = InMemorySaver()
super_agent = create_deep_agent(
    model=model,
    system_prompt='You are a weather expert. You have access to a specialist weather agent that can get the current weather conditions for a given location. Route to the weather agent when you need to get the current weather conditions for a given location.',
    subagents=[weather_agent],
    checkpointer=super_agent_ckpt,
)


## Invoke Agent and Handle Interrupt

This cell demonstrates the interrupt workflow:

- **Thread Configuration**: Sets up a thread ID for stateful conversation tracking
- **Agent Invocation**: Invokes the super agent with a weather query
- **Interrupt Handling**: The weather subagent's `before_model` middleware inspects the most recent `ToolMessage` and pauses whenever `get_current_weather` returns so a human can review the tool output before the next LLM call
- **Interrupt Inspection**: Extracts and prints the interrupt payload to show what information is available for approval


In [4]:

thread_config = {'configurable': {'thread_id': '1'}}

initial_result = super_agent.invoke(
    {'messages': [{'role': 'user', 'content': 'What is the current weather in San Francisco?'}]},
    config=thread_config,
)
interrupt_result = initial_result['__interrupt__'][0]
print('Super agent interrupt payload:', interrupt_result.value)

Super agent interrupt payload: {'prompt': 'Approve or edit the latest weather lookup result.', 'tool_call_id': 'call_GztCZhZ0RB4tqvQbVuKFGnUD', 'tool_name': 'get_current_weather', 'tool_output': '{"query": "San Francisco current weather", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco, California, USA", "url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1762552110, \'localtime\': \'2025-11-07 13:48\'}, \'current\': {\'last_updated_epoch\': 1762551900, \'last_updated\': \'2025-11-07 13:45\', \'temp_c\': 19.4, \'temp_f\': 66.9, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 4.7, \'wind_kph\': 7.6, \'wind_degree\': 271, 

## Resume Agent After Approval

This cell demonstrates resuming the agent after approval (or rejection):

- **Resume Command**: Creates a `Command` with `resume` data that includes a `status` ("approved" or "rejected") and an optional `message` (required if rejecting) that will be inserted as a `HumanMessage` after the tool message
- **Agent Resume**: Invokes the super agent again with the resume command, continuing from where it was interrupted
- **Result Extraction**: Extracts tool messages, the inserted review message, and the final agent response to show the complete workflow result


In [5]:
review_message = {
    'status': 'approved',
    'message': '',
}
resume_cmd = Command(resume=review_message)
resume_result = super_agent.invoke(resume_cmd, config=thread_config)
resume_result['messages'][-1].pretty_print()


================================== Ai Message ==================================

Current weather in San Francisco, CA:
- Temperature: 66.9°F (19.4°C)
- Conditions: Partly cloudy
- Wind: 4.7 mph from the West, gusts up to 6.6 mph
- Humidity: 76%
- Visibility: 9.0 miles (16.0 km)
- Weather alerts: None active
- Last updated: 1:45 PM PT

Would you like any additional details (dew point, pressure, or hourly forecast)?
